In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} pygraphviz

## Ülesande püstitus:

Probleem advmod-i saab puust eemaldada vaid siis kui see on vaba laiend
See kas advmod on vaba laiend on määratud verbi ja kontekstiga
Kui me ignoreerime konteksti, siis on võimalikud järgnevad  asjad

a) verbil on alarti advmod seotud laiend ---> verb ja advmod on alati kollokatsioonis
b)  verbil on advmod küllat tihti vaba laiend --> verb on tihti ilma advmod kollokatsioonita

Seda saab statistiliselt lihtsalt mõõta kui sa moodustad mulle  
verb advmod kollokatsioonid
verb no-advmod kollokatsioonid

Boonus oleks kui sa vaataksid ühendverbe eraldi

# Verbid, ühendverbid ja kõik sellega vahetult seotud deprel="amod" tipud  (korpuse tsv faili alusel)

## Vajalikud teegid

In [2]:
from __future__ import print_function

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from collections import defaultdict
import matplotlib.pyplot as plt
import random

#import etnltk
#from estnltk.storage.postgres import PostgresStorage

import networkx as nx
import sqlite3
import sys


## Abifunktsioonid

In [3]:
class graphFunctions:

    # kahe listi ühisosa 
    def intersection(a, b):
        return list(set(a).intersection(b))

    # tipu leidmine atribuudi väärtuse järgi
    def get_nodes_by_attributes(G,  attrname, attrvalue ):
        nodes = defaultdict(list)
        {nodes[v].append(k) for k, v in nx.get_node_attributes(G,attrname).items()}
        if attrvalue in nodes:
            return dict(nodes)[attrvalue]
        return []

    # graafi joonistamine 
    # tipp - lemma
    # serv - deprel
    def drawGraph(G):
        pos = graphviz_layout(G, prog='dot')
        labels = nx.get_node_attributes(G, 'lemma')
        nx.draw(G, pos, cmap = plt.get_cmap('jet'),labels=labels, with_labels=True)
        edge_labels = nx.get_edge_attributes(G, 'deprel')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        plt.show()


In [4]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

### Baasi  (sqlite3)  ettevalmistamine täitmiseks

Luuakse tabelid:
* collections_processed - salvestatakse viimane salvestatud collection Id
* {TABLENAME} - tabel kollokatsioonidega
 * verb_lemma text
 * verb_pos text
 * compound_lemma text
 * compound_pos text
 * advmod_lemma text
 * advmod_pos text
 * total integer
 * example1 text
 * example2 text
 * example3 text

Luuakse indeksid:
* collections_processed_uniq
* {TABLENAME}_col1_col2_unique

In [17]:
def prepCollDb():
    global TABLENAME, cursor, conn

    cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections_processed
                      (tablename text, lastcollection integer);
                      """)
    
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS collections_processed_uniq ON collections_processed(tablename);
    """)
    
    #tsv failist lugemise korral loome tabeli alati nullist
    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, 0, 0,) )
    
 
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS {TABLENAME}
                      (verb_lemma text
                      , verb_pos text
                      , compound_lemma text
                      , compound_pos text
                      , advmod_lemma text
                      , advmod_pos text
                      , total integer
                      , example1 text
                      , example2 text
                      , example3 text);
                      """)

    #tsv failist lugemise korral loome tabeli alati nullist
    cursor.execute(f"""DELETE FROM {TABLENAME};""")

    
    INDEXNAME = f'{TABLENAME}_col1_col2_unique'
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}
        ON {TABLENAME}(
            verb_lemma
            , verb_pos
            , compound_lemma
            , compound_pos
            , advmod_lemma
            , advmod_pos
            );
        """)
    
    conn.commit()


Tulemuse salvestamine baasi (sqlite3)

In [6]:
def saveCollToDb(collocations, examples, lastcollection):
    
    global TABLENAME, cursor, conn
    sqlColls = []

    for key in collocations.keys():
        example1 = None
        example2 = None
        example3 = None

        if len(examples[key])>0:
            example1 = examples[key][0]
        if len(examples[key])>1:
            example2 = examples[key][1]
        if len(examples[key])>2:
            example3 = examples[key][2]
        
        
            
        sqlColls.append( (key[0]
                          , key[1]
                          , key[2]
                          , key[3]
                          , key[4]
                          , key[5]
                          , collocations[key]
                          , example1
                          , example2
                          , example3
                          , collocations[key]
                          , example1
                          , example2
                          , example3,) )

    cursor.executemany(f"""
        INSERT INTO {TABLENAME} VALUES (?,?,?,?,?,?,?,?,?,?)
        ON CONFLICT(
            verb_lemma
            , verb_pos
            , compound_lemma
            , compound_pos
            , advmod_lemma
            , advmod_pos
            )
        DO UPDATE SET total=total+?
            ,  example1= ?
            ,  example2= ?
            ,  example3= ?
            ;""", sqlColls)

    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, lastcollection, lastcollection,) )
    
    conn.commit()
    eprint(f'andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - {lastcollection}' )
    

### advmod - sõltuvuste leidmine 

* Verb ja advmod või advmod puudumine



In [7]:
def extract_advmod_verb_extended(G, collocations, examples, sentence):
    """
        Eraldame lausest kõik verb - advmod kollokatsioonid
        Kui sellist kollokatsiooni ei leitud, siis paneme kirja verb - None kollokatsiooni
        advmod puhul paneme kirja ka käände ning lemma?
        
    """

    # lyhim tee tippude vahel
    path = nx.all_pairs_shortest_path_length(G)
    path_reversed = nx.all_pairs_shortest_path_length(G.reverse())

    # kauguste maatriksid
    dpath = {x[0]:x[1] for x in path}
    dpath_reversed = {x[0]:x[1] for x in path}
    
    #kõik verbid
    verb_nodes = graphFunctions.get_nodes_by_attributes(G,  attrname = 'pos', attrvalue =  'V')
    
    #kõik advmod
    advmod_nodes = graphFunctions.get_nodes_by_attributes(G,  attrname = 'deprel', attrvalue =  'advmod')
    
    #kõik compound:prt
    compound_nodes = graphFunctions.get_nodes_by_attributes(G,  attrname = 'deprel', attrvalue =  'compound:prt')
    
    #eprint('eraldame tipud vajalike parameetritega ')
    
    #vaatame verbide kaupa:
    
    for verb in verb_nodes:
        kids = [k for k in dpath[verb] if dpath[verb][k] == 1]
        n_advmods = graphFunctions.intersection(kids, advmod_nodes)
        n_compounds = graphFunctions.intersection(kids, compound_nodes)
        
        # kui pole, siis lisame None
        if not len(n_advmods):
            n_advmods.append(None)
        if not len(n_compounds):
            n_compounds.append(None)
        
        for compound in n_compounds:
            #paneme '' mitte None, sest sqllite ei rajenda unique constrainti muidu
            cp_lemma = ''
            cp_pos = ''
            if compound:
                cp_lemma = G.nodes[compound]['lemma']
                cp_pos = G.nodes[compound]['pos']
                
            for advmod in n_advmods:
                advmod_lemma = ''
                advmod_pos = ''
                if advmod:
                    advmod_lemma = G.nodes[advmod]['lemma']
                    advmod_pos = G.nodes[advmod]['pos']
            
                key = ( G.nodes[verb]['lemma'], G.nodes[verb]['pos'], cp_lemma, cp_pos, advmod_lemma, advmod_pos , )
                if not key in collocations:
                    collocations[key] = 0
                if not key in examples:
                    examples[key] = []
                collocations[key] += 1
                examples[key].append(sentence)
                if len(examples[key]) > 3:
                    del(examples[key][random.randint(0, 2)])
                
        
    return (collocations, examples)  
   

## Muutujad

**Korpuse TSV fail**

Lähte TSV-faili ja DB tabeli nimi, kuhu tulemus salvestatakse (TSV genereerimise kood on failis **collect_texts_db_tsv.ipynb**)

In [8]:
collectionName = 'koondkorpus_base' # 'koondkorpus_base_subset_of_5000_v2'
corporafile = f'koondkorpus_base_v2_20220216.tsv'

**Kollokatsiooni tüüp**

In [9]:
TYPE = 'verb_compound_advmod'

**Tabeli nimi andmebaasis**

In [10]:
TABLENAME = f'{TYPE}_{collectionName}'

**Kollektsioonide arv**, mille kaupa salvestatakse vahepealne tulemus andmebaasi.

In [11]:
BATCHSIZE = 1000

## Andmebaasi loomine ja ette valmistamine

In [12]:
conn = sqlite3.connect(f"{TYPE}_collocations.db") #
cursor = conn.cursor()
prepCollDb()

## Kollokatsioonide leidmine

Andmed loetakse TSV failist. Igast lausest tehakse *networkx* suunatud graaf. Graafi servadeks on süntaksipuu 
head->child seos. Tippudeks on lause sõnad. 
Tipu omadused on:
* *id* - sõna id
* deprel
* lemma
* pos


In [13]:

unsaved = 0
collocations = {}
examples = {}
word_id = 0
count = 0
current_sentence = []

#
G = None
colId = None
prevCol = 0
with open(corporafile) as f:
    for line in f:
        count +=1
        line = line.strip('\r\n')
        row = line.split('\t')
        if not len(row) == 9:
            continue
        data = {}
        prevCol = colId
        (colId, data['start'], data['id'], data['text'], data['lemma'], data['upostag'], data['deprel'], data['head'], data['feat']) = row
        colId = int(colId)
        for k in ('id', 'start', 'head'):
            data[k] = int(data[k])

        word_id +=1
        #sentence start
        if data['id'] == 1:
            if not G==None:
                current_sentence_text = ' '.join([s['text'] for s in current_sentence])
                #print (current_sentence_text)
                #graphFunctions.drawGraph(G)
                (collocations, examples) = extract_advmod_verb_extended(G, collocations, examples, current_sentence_text)
                unsaved = 1
            current_sentence = []
            #count+=1
            word_id +=1
            G = nx.DiGraph()

            if not prevCol ==colId and not colId==0 and not colId%BATCHSIZE:
                saveCollToDb(collocations, examples, colId)
                collocations = {}
                unsaved = 0

        #paneme graafi kokku
        G.add_node(word_id, id=data['id'], lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'])
        G.add_edge(word_id - data['id'] + data['head'], word_id, deprel = data['deprel'])
        current_sentence.append(data)
    
    #viimane lause
    current_sentence_text = ' '.join([s['text'] for s in current_sentence])
    (collocations, examples) = extract_advmod_verb_extended(G, collocations, examples, current_sentence_text)
    saveCollToDb(collocations, examples, colId)


andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 1000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 2000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 3000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 4000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 5000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 6000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 7000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 8000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 9000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 10000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 11000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 12000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 13000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 14000
andmebaasi salvestatud kollokatsioonid kollektsioonidest:

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 122000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 123000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 124000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 125000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 126000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 127000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 128000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 129000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 130000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 131000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 132000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 133000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 134000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 135000
andmebaasi salvestatud kollokatsio

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 241000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 242000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 243000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 244000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 245000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 246000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 247000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 248000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 249000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 250000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 251000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 252000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 253000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 254000
andmebaasi salvestatud kollokatsio

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 360000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 361000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 362000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 363000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 364000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 365000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 366000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 367000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 368000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 369000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 370000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 371000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 372000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 373000
andmebaasi salvestatud kollokatsio

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 479000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 480000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 481000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 482000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 483000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 484000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 485000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 486000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 487000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 488000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 489000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 490000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 491000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 492000
andmebaasi salvestatud kollokatsio

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 598000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 599000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 600000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 601000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 602000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 603000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 604000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 605000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 606000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 607000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 608000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 609000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 610000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 611000
andmebaasi salvestatud kollokatsio

### Andmebaasi indeksite lisamine


Tehakse kõige viimasena, et andmete sisestamine andmebaasi oleks kiirem.

In [14]:

indexesQ = [
    f'CREATE INDEX IF NOT EXISTS "verb_lemma" ON "{TABLENAME}" ("verb_lemma");'
    , f'CREATE INDEX IF NOT EXISTS "compound_lemma" ON "{TABLENAME}" ("compound_lemma");'
    , f'CREATE INDEX IF NOT EXISTS "advmod_lemma" ON "{TABLENAME}" ("advmod_lemma");'
    , f'CREATE INDEX IF NOT EXISTS "verb_pos" ON "{TABLENAME}" ("verb_pos");'
    , f'CREATE INDEX IF NOT EXISTS "compound_pos" ON "{TABLENAME}" ("compound_pos");'
    , f'CREATE INDEX IF NOT EXISTS "advmod_pos" ON "{TABLENAME}" ("advmod_pos");'
    , f'CREATE INDEX IF NOT EXISTS "total" ON "{TABLENAME}" ("total" DESC);']

for q in indexesQ: cursor.execute(q)

In [15]:
cursor.execute(f"SELECT count(*) FROM {TABLENAME}")
all_collocations = cursor.fetchall()

In [16]:
all_collocations

[(1087318,)]